# Lemmatize Text From Twitter

## Step 1: Import libraries

In [1]:
import tensorflow as tf
import torch
import malaya
import Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import simplemma
import string
import re
import snscrape.modules.twitter as sntwitter
import pandas as pd
from cleantext import clean #to remove emoji in text
from googletrans import Translator

C:\Users\Jason\anaconda3\lib\site-packages\malaya_boilerplate\frozen_graph.py:35: UserWarning: Cannot import beam_search_ops from Tensorflow Addons, ['malaya.jawi_rumi.deep_model', 'malaya.phoneme.deep_model', 'malaya.rumi_jawi.deep_model', 'malaya.stem.deep_model'] will not available to use, make sure Tensorflow Addons version >= 0.12.0
  warnings.warn(
C:\Users\Jason\anaconda3\lib\site-packages\malaya_boilerplate\frozen_graph.py:38: UserWarning: check compatible Tensorflow version with Tensorflow Addons at https://github.com/tensorflow/addons/releases
  warnings.warn(
C:\Users\Jason\anaconda3\lib\site-packages\malaya\tokenizer.py:208: FutureWarning: Possible nested set at position 3372
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
C:\Users\Jason\anaconda3\lib\site-packages\malaya\tokenizer.py:208: FutureWarning: Possible nested set at position 3890
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


## Step 2: Scraping

In [2]:
# Created a list to append all tweet attributes(data)
tweets_content = []

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper('Anwar Ibrahim').get_items()):
    if i>=10:
        break
    tweets_content.append(tweet.content)
tweets_content

['@JPenerangan @anwaribrahim Usah snaraikan bab tamatkan lantikan politik dlm GLC. Ia akan diteruskan spt dijelaskan ole PM, tp dgn org politik parti dia kot.',
 '@PDRMsia @anwaribrahim @rafiziramli take actions now https://t.co/lKEy80ea6q',
 '@anwaribrahim semasa menjadi Menteri Pertanian 👍🏻🇲🇾\n#pmx #kerajaanydpa https://t.co/9DiM39hqkv',
 'Kita mungkin saksikan samada @anwaribrahim &amp; @rafiziramli mampu pulihkan ekonomi buruk yg di tinggalkan oleh @pakatanharapan_ &amp; @PPBMofficial @PASPusat , hutang lebih RM1.0 trillion .\nGood luck korang²',
 "It's happening again!\n\n#anwaribrahim #whowantstobeamillionaire #PM10malaysia\n\nhttps://t.co/48hHHinarh",
 '@anwaribrahim boss, tambun needs you to redo the roads. Build more interconnected roads in IPOH, especially the suburban areas. Take note thank you. https://t.co/23HqwX9nqD',
 '@BN4everTN5O @anthonyloke @anwaribrahim DSAI mohon',
 '@FreeMsian I only hope @fahmi_fadzil n @anwaribrahim can remove tm ceo ,bod plus dnb..',
 'It’s a n

## Step 3: Preprocessing

In [3]:
content_filtered = [] #cleaned
token_raw = []
tokenizer = malaya.tokenizer.Tokenizer()

def append_token(token):
    for i in token:
        token_raw.append(i)

for i in tweets_content:
    append_token(tokenizer.tokenize(i))
        
for i in token_raw:
    #Covert all letters to lowercase
    i.lower()
    #Remove emoji in text
    i = clean(i, no_emoji=True)
    #Remove all @mention,newlines,hyperlink
    i = re.sub("#[A-Za-z0-9_]+|@[A-Za-z0-9_]+|\n|http\S+|\"","",i)
    #Remove punctuation
    i = i.translate(str.maketrans("", "", string.punctuation))
    #Remove no tweet content from list
    if i == "":
        continue
    else:
        content_filtered.append(i)

content_filtered

['usah',
 'snaraikan',
 'bab',
 'tamatkan',
 'lantikan',
 'politik',
 'dlm',
 'glc',
 'ia',
 'akan',
 'diteruskan',
 'spt',
 'dijelaskan',
 'ole',
 'pm',
 'tp',
 'dgn',
 'org',
 'politik',
 'parti',
 'dia',
 'kot',
 'take',
 'actions',
 'now',
 'semasa',
 'menjadi',
 'menteri',
 'pertanian',
 'kita',
 'mungkin',
 'saksikan',
 'samada',
 'mampu',
 'pulihkan',
 'ekonomi',
 'buruk',
 'yg',
 'di',
 'tinggalkan',
 'oleh',
 'hutang',
 'lebih',
 'rm10',
 'trillion',
 'good',
 'luck',
 'korang2',
 'its',
 'happening',
 'again',
 'boss',
 'tambun',
 'needs',
 'you',
 'to',
 'redo',
 'the',
 'roads',
 'build',
 'more',
 'interconnected',
 'roads',
 'in',
 'ipoh',
 'especially',
 'the',
 'suburban',
 'areas',
 'take',
 'note',
 'thank',
 'you',
 'dsai',
 'mohon',
 'i',
 'only',
 'hope',
 'n',
 'can',
 'remove',
 'tm',
 'ceo',
 'bod',
 'plus',
 'dnb',
 'it',
 's',
 'a',
 'national',
 'asset',
 'yet',
 'it',
 's',
 'sold',
 'for',
 'a',
 'song',
 'masih',
 'mahal',
 'kalau',
 'berbanding',
 'dengan

## Step 4: Normalization

In [4]:
bm_dict = pd.read_csv('BM_dict.csv')
bm_dict.head(20)

,rojak,actual
0,abeh,habis
1,abes,habis
2,abih,habis
3,abg,adik
4,ade,ada
5,ader,ada
6,ado,ada
7,hado,ada
8,adk,adik
9,adeq,adik


### First Normalize

In [5]:
content_normalized1 = []

for i in content_filtered:
    if (bm_dict['rojak'] == i).any():
        str(content_normalized1.append(bm_dict[bm_dict['rojak'] == i]['actual'].to_string(index = False)))
    else:
        content_normalized1.append(i)

content_normalized1

['usah',
 'snaraikan',
 'bab',
 'tamatkan',
 'lantikan',
 'politik',
 'dalam',
 'glc',
 'ia',
 'akan',
 'diteruskan',
 'seperti',
 'dijelaskan',
 'ole',
 'perdana menteri',
 'tapi',
 'dengan',
 'orang',
 'politik',
 'parti',
 'dia',
 'mungkin',
 'take',
 'actions',
 'now',
 'semasa',
 'menjadi',
 'menteri',
 'pertanian',
 'kita',
 'mungkin',
 'saksikan',
 'samada',
 'mampu',
 'pulihkan',
 'ekonomi',
 'buruk',
 'yang',
 'di',
 'tinggalkan',
 'oleh',
 'hutang',
 'lebih',
 'rm10',
 'trillion',
 'good',
 'luck',
 'korang2',
 'its',
 'happening',
 'again',
 'boss',
 'tambun',
 'needs',
 'you',
 'to',
 'redo',
 'the',
 'roads',
 'build',
 'more',
 'interconnected',
 'roads',
 'in',
 'ipoh',
 'especially',
 'the',
 'suburban',
 'areas',
 'take',
 'note',
 'thank',
 'you',
 'dato seri anwar ibrahim',
 'mohon',
 'i',
 'only',
 'hope',
 'n',
 'can',
 'remove',
 'tm',
 'ceo',
 'bod',
 'plus',
 'dnb',
 'it',
 's',
 'a',
 'national',
 'asset',
 'yet',
 'it',
 's',
 'sold',
 'for',
 'a',
 'song',
 '

### Second Normalize 

In [6]:
corrector = malaya.spelling_correction.probability.load()
normalizer = malaya.normalize.normalizer(corrector)

content_normalized2 = []

for i in content_normalized1:
    content_normalized2.append(normalizer.normalize(i)["normalize"])
       
content_normalized2

['usah',
 'senaraikan',
 'bab',
 'tamatkan',
 'lantikan',
 'politik',
 'dalam',
 'glc',
 'ia',
 'akan',
 'diteruskan',
 'seperti',
 'dijelaskan',
 'ole',
 'perdana menteri',
 'tetapi',
 'dengan',
 'orang',
 'politik',
 'parti',
 'dia',
 'mungkin',
 'take',
 'actions',
 'now',
 'semasa',
 'menjadi',
 'menteri',
 'pertanian',
 'kita',
 'mungkin',
 'saksikan',
 'samada',
 'mampu',
 'pulihkan',
 'ekonomi',
 'buruk',
 'yang',
 'di',
 'tinggalkan',
 'oleh',
 'hutang',
 'lebih',
 'sepuluh ringgit',
 'trillion',
 'good',
 'luck',
 'kamu semua-kamu semua',
 'its',
 'happening',
 'again',
 'boss',
 'tambun',
 'needs',
 'you',
 'kepada',
 'redo',
 'the',
 'roads',
 'build',
 'more',
 'interconnected',
 'roads',
 'ini',
 'ipoh',
 'especially',
 'the',
 'suburban',
 'areas',
 'take',
 'note',
 'thank',
 'you',
 'dato seri anwar ibrahim',
 'mohon',
 'saya',
 'only',
 'hope',
 'dan',
 'can',
 'remove',
 'temu',
 'ceo',
 'bod',
 'plus',
 'donbo',
 'itu',
 'seperti',
 'a',
 'national',
 'asset',
 'yet'

## Step 5: Translation

In [7]:
from googletrans import Translator
translator = Translator()

content_translated = []

for i in content_normalized2:
    translate = translator.translate(i, src='en', dest = 'id')
    content_translated.append(translate.text.lower())

content_translated

ReadTimeout: The read operation timed out

## Step 6: Lemmatization

In [ ]:
content_lemma = []
sastrawi = malaya.stem.sastrawi()

for i in content_translated:
    content_lemma.append(sastrawi.stem(i).lower())
       
content_lemma

## Output Result

In [ ]:
raw_lemma_df = pd.DataFrame(content_filtered)
norma_df = pd.DataFrame(content_normalized2)
trans_df = pd.DataFrame(content_translated)
lemma_df = pd.DataFrame(content_lemma)

raw_lemma_df.rename(columns = {0:'Raw'}, inplace = True)
raw_lemma_df['Normalized'] = norma_df
raw_lemma_df['Translated'] = trans_df
raw_lemma_df['Lemmatized'] = lemma_df

raw_lemma_df.sample(50)

# Lemmatize From User Input

In [ ]:
def lemmatization_input(userinput):
    token_raw = []
    content_filtered = []
    content_normalized1 = []
    content_normalized2 = []
    content_translated = []
    content_lemma = []
    tokenizer = malaya.tokenizer.Tokenizer()
    corrector = malaya.spelling_correction.probability.load()
    normalizer = malaya.normalize.normalizer(corrector)
    translator = Translator()
    sastrawi = malaya.stem.sastrawi()
    bm_dict = pd.read_csv('BM_dict.csv')

    
    def chk_content(token_raw,content_filtered):
        if len(token_raw) != 0:
            token_raw.clear()
            content_filtered.clear()
            content_normalized1.clear()
            content_normalized2.clear()
            content_translated.clear()
            content_lemma.clear()
    
    def append_token(token):
        for i in token:
            token_raw.append(i)
    
    chk_content(token_raw,content_filtered)
    append_token(tokenizer.tokenize(userinput))

    for i in token_raw:
        #Covert all letters to lowercase
        i.lower()
        #Remove emoji in text
        i = clean(i, no_emoji=True)
        #Remove all @mention,newlines,hyperlink
        i = re.sub("#[A-Za-z0-9_]+|@[A-Za-z0-9_]+|\n|http\S+|\"","",i)
        #Remove punctuation
        i = i.translate(str.maketrans("", "", string.punctuation))
        if i == "":
            continue
        else:
            content_filtered.append(i)
    
    for i in content_filtered:
        if (bm_dict['rojak'] == i).any():
            str(content_normalized1.append(bm_dict[bm_dict['rojak'] == i]['actual'].to_string(index = False)))
        else:
            content_normalized1.append(i)
    
    for i in content_normalized1:
        content_normalized2.append(normalizer.normalize(i)["normalize"])
        
    for i in content_normalized2:
        translate = translator.translate(i, src='en', dest = 'id')
        content_translated.append(translate.text.lower())
    
    for i in content_translated:
        content_lemma.append(sastrawi.stem(i).lower())

    return content_lemma

In [ ]:
userinput = input("Please enter sentence: ")
lemmatization_input(userinput)